# Problem 
A farmer has soil that can 'good', 'mediocre', or 'bad.The farmer can then sell this soil for prices that depend on quality. After each year, the land will transition to a different state with certain probabilities listed below. However, the farmer can choose to purchase fertilizer to change the probability of next year's soil outcome. Fertilizer comes at varying prices depending on the quality of the land. 

Q1) Is it worth it to buy fertilizer all the time?  
Q2) If not, when is it worth it to purchase fertilizer? 

In [138]:
import numpy as np

In [2]:
class MarkovChain(object):
    def __init__(self, transition_prob):
        """
        Initialize the MarkovChain instance.
 
        Parameters
        ----------
        transition_prob: dict
            A dict object representing the transition 
            probabilities in Markov Chain. 
            Should be of the form: 
                {'state1': {'state1': 0.1, 'state2': 0.4}, 
                 'state2': {...}}
        """
        self.transition_prob = transition_prob
        self.states = list(transition_prob.keys())
 
    def next_state(self, current_state):
        """
        Returns the state of the random variable at the next time 
        instance.
 
        Parameters
        ----------
        current_state: str
            The current state of the system.
        """
        return np.random.choice(
            self.states, 
            p=[self.transition_prob[current_state][next_state] 
               for next_state in self.states]
        )
 
    def generate_states(self, current_state, no=10):
        """
        Generates the next states of the system.
 
        Parameters
        ----------
        current_state: str
            The state of the current random variable.
 
        no: int
            The number of future states to generate.
        """
        future_states = []
        for i in range(no):
            next_state = self.next_state(current_state)
            future_states.append(next_state)
            current_state = next_state
        return future_states

In [105]:
def profit_calculator(price_list, states):
    profit = 0 
    for state in states: 
        profit += price_list[state]
    return profit

def cost_calculator(price_list, states): 
    cost = 0 
    for state in states: 
        cost -= price_list[state]
    return cost

In [ ]:
# instantiate transition probabilities and cost/profit optimization parameters. 
fertilizer_cost = {'good': 1000, 'mediocre': 2000, 'bad': 10000}
soil_sell = {'good': 10000, 'mediocre': 6000, 'bad': 2000 }

no_fertilizer = {'good': {'good': 0.2, 'mediocre': 0.5, 'bad': 0.3 },
                 'mediocre': {'good': 0, 'mediocre': 0.5, 'bad': 0.5 },
                 'bad': {'good': 0, 'mediocre': 0, 'bad': 1 },
                }

fertilizer = {'good': {'good': 0.3, 'mediocre': 0.6, 'bad': 0.1 },
              'mediocre': {'good': 0.2, 'mediocre': 0.6, 'bad': 0.2 },
              'bad': {'good': 0.1, 'mediocre': 0.4, 'bad': .5 },
             }

In [ ]:
# Calculated expected values for each transition probability
no_fert = np.vstack([np.array(list(no_fertilizer['good'].values())), 
           np.array(list(no_fertilizer['mediocre'].values())), 
           np.array(list(no_fertilizer['bad'].values()))])

fert = np.vstack([np.array(list(fertilizer['good'].values())), 
           np.array(list(fertilizer['mediocre'].values())), 
           np.array(list(fertilizer['bad'].values()))])

fertilize_expected_values = np.matmul(fert, np.array(list(soil_sell.values()))) - np.array(list(fertilizer_cost.values()))
no_fertilize_expected_values = np.matmul(no_fert, np.array(list(soil_sell.values())))

print("if fertilize EV: " + str(dict(zip(list(fertilizer.keys()),fertilize_expected_values.tolist()))))
print("if no fertilize EV: " + str(dict(zip(list(fertilizer.keys()),no_fertilize_expected_values.tolist()))))
print('therefore, only fertilize if good.')

In [106]:
# build transition probability and cost for fertilize only when good
fertilize_only_good = {'good': {'good': 0.3, 'mediocre': 0.6, 'bad': 0.1 },
                     'mediocre': {'good': 0, 'mediocre': 0.5, 'bad': 0.5 },
                     'bad': {'good': 0, 'mediocre': 0, 'bad': 1 },
                     }
gfertilizer_cost = {'good': 1000, 'mediocre': 0, 'bad': 0}


In [104]:
# instantiate markov chains for each one of the transition probabilities.
fertilize_chain = MarkovChain(fertilizer)
g_fertilize_chain = MarkovChain(fertilize_only_good)
no_fertilize_chain = MarkovChain(no_fertilizer)

In [137]:
# simulate 100,000 trials and calculate cost and profit 

f_states = fertilize_chain.generate_states(current_state='good', no=100000) #always fertilize
gf_states = g_fertilize_chain.generate_states(current_state='good', no=100000) #only fertilize if good
nf_states = no_fertilize_chain.generate_states(current_state='good', no=100000) #never fertilize


nf_profit = profit_calculator(soil_sell, nf_states)
gf_profit = profit_calculator(soil_sell, gf_states) + cost_calculator(gfertilizer_cost, gf_states)
f_profit = profit_calculator(soil_sell, f_states) + cost_calculator(fertilizer_cost, f_states)


print("after 100,000 cycles:")
print(str(dict(zip(['never_fertilize', 'fertilize_good', 'always_fertilize'],[nf_profit, gf_profit, f_profit]))))


after 100,000 cycles:
{'never_fertilize': 200000000, 'fertilize_good': 200033000, 'always_fertilize': 183486000}
